<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/feature%2Fibd3mo/notebooks/ibd_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking Inspired by IBD's EPS Rating
#### References
- [Growth Stocks And How To Analyze Earnings Growth: Using IBD's EPS Rating](https://www.investors.com/how-to-invest/investors-corner/growth-stocks-and-how-to-analyze-earnings-growth-using-ibd-eps-rating/)
  - takes a company's two most recent year-over-year changes in quarterly earnings, and the profit of the past three to five years, then compares those results with the thousands of stocks in IBD's database.
- [How To Invest: Check This Rating Before Buying Your Next Stock](https://www.investors.com/how-to-invest/investors-corner/how-to-invest-earnings-growth/)

### Install and Config

#### Install Required Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git to /tmp/pip-req-build-f5y3g5gn
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-f5y3g5gn
  Resolved https://github.com/yorkjong/vistock.git to commit 0028e90b9166d8898fe87c0ff10730a2903a87e4
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.6.0-py3-none-any.whl size=87239 sha256=1f0f3f0d91ba7c6a180c378e30358ce03ad0af3aa7162801b8a9cee6d3ba25ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-n4lqwff8/wheels/1f/ff/6c/9ec0adab3f1af71ff9ba3e4eb52d0f3849d3a695b812dc509a
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.3 MB/s eta 0:00:00


#### Setup and Configuration

In [5]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [6]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [16]:
# @title GitHub
import base64
import requests
import pandas as pd
from io import StringIO


class GitHub:
    def __init__(self, repo_owner, repo_name, token, dir='', branch='main'):
        base = 'https://api.github.com/repos'
        dir = dir.strip('/')
        if dir:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch
        self.token = token

    def _request(self, method, url, headers=None, params=None, json=None):
        response = requests.request(method, url, headers=headers,
                                    params=params, json=json)
        if response.status_code in [200, 201]:
            return response.json()
        elif response.status_code == 404:
            return None
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def file_exists(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def list_filenames(self, dir_path=''):
        url = f'{self.base_url}/{dir_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            #print(f"Request failed: {response.status_code} - {response.json()}")
            return []

    def download_file(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }

        file_info = self._request('GET', url, headers=headers,
                                  params={'ref': self.branch})
        if file_info:
            response = requests.get(file_info['download_url'])
            if response.status_code == 200:
                return StringIO(response.text)
            else:
                print(f"Failed to download file: "
                      f"{response.status_code} - {response.text}")
        else:
            print(f"File '{file_path}' does not exist. Cannot download.")
        return None

    def download_csv(self, file_path):
        file_content = self.download_file(file_path)
        if file_content:
            return pd.read_csv(file_content)
        else:
            return pd.DataFrame()

    def upload_file(self, file_path, content):
        url = f'{self.base_url}/{file_path}'

        # Encode the content to base64
        encoded_content = base64.b64encode(content.encode()).decode()
        payload = {
            'message': 'Uploading file',
            'content': encoded_content,
            'branch': self.branch
        }

        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Check if the file already exists to get the current sha
        file_info = self._request('GET', url, headers=headers,
                                  params={'ref': self.branch})
        # If the file exists, get the current SHA
        if file_info:
            payload['sha'] = file_info.get('sha')

        # PUT request to create or update the file
        self._request('PUT', url, headers=headers, json=payload)

    def upload_df_as_csv(self, file_path, df):
        """Upload a DataFrame to a CSV file."""
        if not file_path.endswith('.csv'):
            file_path += '.csv'
        csv_content = df.to_csv(index=False)
        self.upload_file(file_path, csv_content)

    def remove_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Skipping deletion.")
            return

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Fetch the file info to get the SHA needed for deletion
        file_info = self._request('GET', url, headers=headers)
        if file_info:
            payload = {
                'message': 'Deleting file',
                'sha': file_info['sha'],
                'branch': self.branch
            }
            self._request('DELETE', url, headers=headers, json=payload)

#-------------------------------------------------------------------------------

from google.colab import userdata

github = GitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    token=userdata.get('GithubToken.stock-reports'),
    dir='data/ibd_fin',
)

In [8]:
# @title DataFrame Operations

def is_taiwan_stock_df(df):
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    return ticker.isdigit()

def add_name_column(df):
    column_names = df.columns.tolist()
    if 'Name' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    if df.empty:
        return df
    if not is_taiwan_stock_df(df):
        return df
    df['Name'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Name')
    df = df[column_names]   # create a new DataFrame
    df['Name'] = df['Ticker'].apply(tw.stock_name)
    return df


def remove_ticker_suffix(df, ticker_column):
    """
    Remove the '.TW' or '.TWO' suffix from ticker codes in a DataFrame.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    """
    # Apply string replacement for each ticker in the specified column
    df[ticker_column] = df[ticker_column].str.replace('.TWO', '', regex=False)
    df[ticker_column] = df[ticker_column].str.replace('.TW', '', regex=False)


In [9]:
# @title Rank Function

import os
from datetime import datetime

from vistock import ibd_fin as fin
from vistock import tw
from vistock.stock_indices import get_tickers


def remove_failed_tickers(tickers):
    delisted = ['BRK.B', 'LEN.B', 'BF.B', 'UHAL.B', 'BF.A', 'CWEN.A', 'HEI.A']
    invalid = ['GEV', 'SOLV', 'VLTO', 'SW', 'ARM', 'CART', 'AS', 'BIRK', 'VSTS','LOAR', 'ALAB','GRAL', 'SEG']
    invalid += ['00945B.TW', '6928.TW', '6914.TW', '6771.TW', '00944.TW', '8162.TW', '1563.TW', '00946.TW', '00941.TW', '6423.TW', '00940.TW', '00939.TW', '4949.TW', '00943.TW', '8487.TW', '6794.TW', '6949.TW', '4771.TW']
    invalid += ['00936.TW', '6805.TW', '2254.TW', '6658.TW', '00935.TW', '6592B.TW', '6526.TW', '6906.TW', '4736.TW', '00636K.TW', '6968.TWO', '4442.TWO', '6534.TW', '6901.TW', '00934.TW', '00657K.TW', '6472.TW', '2258.TW', '6916.TW', '2762.TW', '6933.TW']
    invalid += ['02001R.TW', '020031.TW', '020039.TW', '020016.TW', '02001L.TW', '020019.TW', '020028.TW', '020020.TW', '02001S.TW', '020018.TW', '020038.TW', '020034.TW', '020011.TW', '020030.TW', '020012.TW', '020036.TW', '020029.TW', '020000.TW', '020015.TW', '020037.TW']
    invalid += ['6890.TW', '00951.TW', '3150.TW', '6957.TW', '00947.TW', '00949.TW']
    invalid += ['6838.TW', '00953B.TW', '00956.TW', '00954.TW']
    return list(set(tickers) - set(delisted) - set(invalid))

def rank(code, out_dir='out'):
    tickers = get_tickers(code)
    #tickers = remove_failed_tickers(tickers)

    rank_stock = fin.financial_metric_ranking(tickers)
    if rank_stock.empty:
        print("Not enough data to generate rankings.")
        return

    rank_stock = add_name_column(rank_stock)

    # Remove the '.TW' or '.TWO' suffix
    remove_ticker_suffix(rank_stock, 'Ticker')

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    filename = f'{code}_stocks_fin_{today}.csv'
    github.upload_df_as_csv(filename, rank_stock)
    rank_stock.to_csv(os.path.join(out_dir, filename), index=False)
    print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock

### EPS RS Rating and Ranking

In [13]:
source = "PHLX Semiconductor" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "Russell 1000", "Russell 2000", "PHLX Semiconductor", "U.S. Listed Stocks"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'Russell 1000': 'RUI',
    'Russell 2000': 'RUT',
    'PHLX Semiconductor': 'SOX',
    'U.S. Listed Stocks': 'U.S.Listed',
}

df = rank(code_from_name[source])
data_table.DataTable(df, include_index=False, num_rows_per_page=20)

[**********************100%**********************]  30 of 30 info downloaded
[**********************100%**********************]  30 of 30 financials downloaded
[**********************100%**********************]  30 of 30 financials downloaded


***
Your "SOX_stocks_fin_20241004.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE,EPS RS Rank (%),RPS RS Rank (%)
23,QRVO,Technology,Semiconductors,101.68,-5.58,46.99,-2.31,103.41,NaN,6751.92,-0.27,-14.27,41.376000,NaN,100.00,86.67
17,MU,Technology,Semiconductors,101.82,-1.40,2.56,0.17,6.77,NaN,413.22,0.70,10.55,22.725000,146.07,96.67,96.67
2,AMD,Technology,Semiconductors,162.85,1.00,-4.12,1.28,7.00,NaN,410.74,0.85,-19.49,14.399000,201.06,93.33,80.00
20,NXPI,Technology,Semiconductors,233.24,0.04,-0.09,-0.13,-0.04,NaN,367.43,10.68,-35.83,51.066000,21.93,90.00,46.67
19,NVDA,Technology,Semiconductors,122.85,0.13,0.23,0.31,1.72,6.28,302.69,2.14,65.04,3.910000,58.37,86.67,100.00
8,GFS,Technology,Semiconductors,38.95,0.17,-1.08,0.11,-0.54,NaN,46.51,1.46,-40.63,12.443000,27.16,83.33,33.33
27,TSM,Technology,Semiconductors,179.48,0.10,-0.06,0.13,0.36,NaN,2.70,5.71,-4.41,58.775623,31.73,80.00,90.00
26,TER,Technology,Semiconductor Equipment & Materials,129.01,1.81,-0.83,-0.08,0.51,NaN,-1.52,3.04,-34.82,17.518000,42.93,76.67,50.00
12,LRCX,Technology,Semiconductor Equipment & Materials,81.11,0.00,0.02,0.08,0.23,0.22,-11.26,2.90,-23.36,11.343000,28.12,73.33,76.67
1,AMAT,Technology,Semiconductor Equipment & Materials,199.59,-0.01,-0.17,0.01,0.11,NaN,-11.91,8.90,-28.32,32.344000,22.69,70.00,60.00


### EPS RS Rating and Ranking for Taiwan Stocks

In [ ]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

df_tw = rank(code_from_name[source])
data_table.DataTable(df_tw, include_index=False, num_rows_per_page=20)

### Remove files in GitHub Repository

In [17]:
# @title CSV Deleter
import re
import ipywidgets as widgets

# Example filenames
with requests_cache.disabled():
    all_filenames = github.list_filenames()

# Function to extract unique dates from filenames
def extract_dates(filenames):
    date_pattern = r'\d{8}'
    dates = set()
    for fn in filenames:
        match = re.search(date_pattern, fn)
        if match:
            dates.add(match.group(0))
    return sorted(dates, reverse=True)  # Sort dates from newest to oldest

# Function to remove a file (replace with your actual implementation)
def remove_file(filename):
    print(f"Removing file: {filename}")
    with requests_cache.disabled():
        github.remove_file(filename)
    all_filenames.remove(filename)

#-------------------------------------------------------------------------------

# Update file selector options based on selected date
def update_file_selector(change):
    def selector_width(filenames):
        max_filename_length = max(len(fn) for fn in filenames)
        return f'{max_filename_length * 10}px'  # 10px width per character

    selected_date = change['new']
    lst_fns = [fn for fn in all_filenames if selected_date in fn]
    file_selector_widget.options = lst_fns
    file_selector_widget.rows = len(file_selector_widget.options)
    file_selector_widget.layout=widgets.Layout(width=selector_width(lst_fns))

# Function to delete selected files
def delete_files(button):
    selected_files = file_selector_widget.value
    for file in selected_files:
        remove_file(file)
    update_widgets()    # Update widgets after deletion

# Update widgets to reflect current state
def update_widgets():
    # Refresh the date selector
    dates = extract_dates(all_filenames)
    selected_date = date_selector_widget.value
    if dates and selected_date not in dates:
        i = date_selector_widget.options.index(selected_date)
        if i > len(dates) - 1:
            selected_date = dates[-1]
        else:
            selected_date = dates[i]
    if dates:
        date_selector_widget.options = dates
        date_selector_widget.value = selected_date
    else:
        file_selector_widget.options = []
        return
    update_file_selector({'new': date_selector_widget.value})

#-------------------------------------------------------------------------------

# Create a widget for selecting dates
def create_date_selector(dates):
    return widgets.Dropdown(
        options=dates,
        value = dates[0] if dates else None,
        description='Date:',
        disabled=False
    )

# Create a widget for selecting files
def create_file_selector(filenames):

    return widgets.SelectMultiple(
        options=[],
        value=[],
        description='Files',
        disabled=False,
    )

# Create widgets
dates = extract_dates(all_filenames)
date_selector_widget = create_date_selector(dates)
file_selector_widget = create_file_selector(all_filenames)
delete_button = widgets.Button(description="Delete Selected Files")
delete_button.on_click(delete_files)

# Initialize the file selector with the latest date
update_widgets()

# Set up the observer to update file selector when date is changed
date_selector_widget.observe(update_file_selector, names='value')

# Display widgets
display(date_selector_widget, file_selector_widget, delete_button)

Dropdown(description='Date:', options=(), value=None)

SelectMultiple(description='Files', options=(), value=())

Button(description='Delete Selected Files', style=ButtonStyle())